# 5-4-如何将文件映射到内存

## 实际案例
1. 在访问某些二进制文件时，希望把文件映射到内存中，可以实现随机访问。(`framebuffer`设备文件，控制屏幕显示)
2. 某些嵌入式设备，寄存器被编址到内存地址空间，我们可以映射`/dev/mem`某范围，去访问这些寄存器
3. 如果多个进程映射同一个文件，还能实现进程通信的目的

In [1]:
!dd if=/dev/zero of=./5-4-test.bin bs=1024 count=1024

1024+0 records in
1024+0 records out
1048576 bytes (1.0 MB, 1.0 MiB) copied, 0.00266273 s, 394 MB/s


In [2]:
!od -x ./5-4-test.bin # 以16进制查看文件

0000000 0000 0000 0000 0000 0000 0000 0000 0000
*
4000000


## python

In [3]:
import mmap

`mmap.mmap?`  
Init signature: mmap.mmap(self, /, *args, **kwargs)
Docstring:     
Windows: mmap(fileno, length[, tagname[, access[, offset]]])

Maps length bytes from the file specified by the file handle fileno,
and returns a mmap object.  If length is larger than the current size
of the file, the file is extended to contain length bytes.  If length
is 0, the maximum length of the map is the current size of the file,
except that if the file is empty Windows raises an exception (you cannot
create an empty mapping on Windows).

Unix: mmap(fileno, length[, flags[, prot[, access[, offset]]]])

Maps length bytes from the file specified by the file descriptor fileno,
and returns a mmap object.  If length is 0, the maximum length of the map
will be the current size of the file when mmap is called.
flags specifies the nature of the mapping. MAP_PRIVATE creates a
private copy-on-write mapping, so changes to the contents of the mmap
object will be private to this process, and MAP_SHARED creates a mapping
that's shared with all other processes mapping the same areas of the file.
The default value is MAP_SHARED.

To map anonymous memory, pass -1 as the fileno (both versions).
File:           /usr/lib/python3.8/lib-dynload/mmap.cpython-38-x86_64-linux-gnu.so
Type:           type
Subclasses:     

---

`Unix: mmap(fileno, length[, flags[, prot[, access[, offset]]]])`

---

`fileno`是`文件描述符`  
在python里可以使用`os.open`或者`open("xxx","xx").fileno()`返回
___

`length`映射长度，长度为0,则整个文件都映射  
___

`offset` 设置映射起始点，必须是页对齐，即必须是整页整页映射（可以使用`mmap.PAGESIZE`查看页大小），即传入的`offset`大小必须是页大小的整数倍
___

`flag` 指定映射模式,`MAP_PRIVATE`,和`MAP_SHARED`，是否可被共享。
___

`prot` 指定文件读写权限`mmap.PROT_EXEC` `mmap.PROT_READ` `mmap.PROT_WRITE`

In [4]:
f = open("./5-4-test.bin","r+b")
m = mmap.mmap(f.fileno(),0)
m.write(b"abc")

3

In [5]:
!od -x ./5-4-test.bin

0000000 6261 0063 0000 0000 0000 0000 0000 0000
0000020 0000 0000 0000 0000 0000 0000 0000 0000
*
4000000


In [6]:
m[0],m[1],m[2],m[:3] # a,b,c

(97, 98, 99, b'abc')

In [7]:
m[1] = 78

In [8]:
m[1]

78

In [9]:
!od -x ./5-4-test.bin

0000000 4e61 0063 0000 0000 0000 0000 0000 0000
0000020 0000 0000 0000 0000 0000 0000 0000 0000
*
4000000


In [10]:
m[:3] = b"\xaa" * 3

## 实例，操作`framebuffer`文件,使显示上面一半变白

使用`sudo fbset`命令查看`framebuffer`文件
```
mode "3200x2000"
    geometry 3200 2000 3200 2000 32
    timings 0 0 0 0 0 0 0
    accel true
    rgba 8/16,8/8,8/0,0/0
endmode
```
---
`32`表示4*8,即4位，也就是rgba占用位数  

In [ ]:
# import mmap
# with open("/dev/fb0","r+b") as f:
#     size = 3200 * 2000 * 4 # 见上面`fbset`命令输出的设备信息
#     m = mmap.mmap(f.fileno(),size)
#     m[:size//2] = b"\xff\xff\xff\x00" * (size // 4 // 2)
#     m.close()

注意使用`Ctrl-Alt-F$` (`$`为数字1,2,3,etc)切换到tty后用`sudo`命令测试